In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import numpy as np
import pandas as pd
import collections as col
import tsfresh as tsf
import scipy as sp

import matplotlib.pyplot as plt

import time

In [4]:
df = pd.read_csv('data/ETHBTC_1517303160000_1547498100000.csv')

In [5]:
df.head(10)

OPEN_TIME      OPEN      HIGH       LOW     CLOSE       VOLUME  \
0  1517303100000  10629400  10673700  10629300  10640100  17785000000   
1  1517303160000  10640100  10643300  10619800  10630100  15433400000   
2  1517303220000  10621100  10640000  10621000  10625000   9354600000   
3  1517303280000  10626200  10634500  10622600  10630800   6746400000   
4  1517303340000  10630800  10639000  10622600  10632600   7650900000   
5  1517303400000  10632600  10632600  10620000  10622400   6602300000   
6  1517303460000  10624100  10655300  10621900  10633100  18857000000   
7  1517303520000  10633100  10648400  10628900  10632200  10156400000   
8  1517303580000  10632600  10649200  10632200  10644200  15437200000   
9  1517303640000  10644000  10690000  10633500  10644300  19295500000   

      CLOSE_TIME   QUOTE_VOL  NUM_TRADES  TAKE_BASE_VOL  TAKE_QUOTE_VOL  EXTRA  
0  1517303159999  1892484646         412    10429400000      1110434538      0  
1  1517303219999  1640768918         276     4288000000       456085864      0  
2  1517303279999   994314138         255     4298600000       457129449      0  
3  1517303339999   716919384         184     3005400000       319476265      0  
4  1517303399999   813139316         182     3471000000       369112330      0  
5  1517303459999   701424997         191     2897600000       307888440      0  
6  1517303519999  2005967577         415     8383700000       892185162      0  
7  1517303579999  1080391056         270     6018500000       640444989      0  
8  1517303639999  1642054242         282     4604000000       490073186      0  
9  1517303699999  2054704543         352    11366500000      1211303174      0

In [6]:
from tsfresh.feature_extraction import feature_calculators as tsf

In [20]:
def get_feats_tsf(data, source='CLOSE', period=5):
    strt = time.time()
    feats = pd.DataFrame({})
    
    feats['mean']                           = data[source].rolling(period).mean()
    feats['std']                            = data[source].rolling(period).std()
    feats['skew']                           = data[source].rolling(period).skew()
    feats['kurtosis']                       = data[source].rolling(period).kurt()
    feats['sum']                            = data[source].rolling(period).sum()
    feats['min']                            = data[source].rolling(period).min()
    feats['max']                            = data[source].rolling(period).max()
    feats['q_0.1']                          = data[source].rolling(period).apply(lambda x: np.quantile(x, 0.1), raw=True)
    feats['q_0.2']                          = data[source].rolling(period).apply(lambda x: np.quantile(x, 0.2), raw=True)
    feats['q_0.3']                          = data[source].rolling(period).apply(lambda x: np.quantile(x, 0.3), raw=True)
    feats['q_0.4']                          = data[source].rolling(period).apply(lambda x: np.quantile(x, 0.4), raw=True)
    feats['q_0.5']                          = data[source].rolling(period).apply(lambda x: np.quantile(x, 0.5), raw=True)
    feats['q_0.6']                          = data[source].rolling(period).apply(lambda x: np.quantile(x, 0.6), raw=True)
    feats['q_0.7']                          = data[source].rolling(period).apply(lambda x: np.quantile(x, 0.7), raw=True)
    feats['q_0.8']                          = data[source].rolling(period).apply(lambda x: np.quantile(x, 0.8), raw=True)
    feats['q_0.9']                          = data[source].rolling(period).apply(lambda x: np.quantile(x, 0.9), raw=True)
    feats['coeffofvar']                     = feats['std']/feats['mean']
    feats['iqr']                            = data[source].rolling(period).apply(lambda x: np.quantile(x, 0.75) - np.quantile(x, 0.25), raw=True)
    feats['hinge']                          = data[source].rolling(period).apply(lambda x: np.quantile(x, 0.75) + np.quantile(x, 0.25), raw=True)
    feats['quartile_coeff_of_dispersion']   = feats['iqr']/feats['hinge']
    feats['range']                          = feats['max'] - feats['min']
    feats['symm_coeff']                     = (feats['mean'] - feats['q_0.5']) / feats['range']
    
    feats['abs_energy']                     = data[source].rolling(period).apply(lambda x: tsf.abs_energy(x), raw=True)
    feats['abs_sum_of_changes']             = data[source].rolling(period).apply(lambda x: tsf.absolute_sum_of_changes(x), raw=True)
    feats['sample_entropy']                 = data[source].rolling(period).apply(lambda x: tsf.sample_entropy(x), raw=True)
    feats['mean_change']                    = data[source].rolling(period).apply(lambda x: tsf.mean_change(x), raw=True)
    feats['mean_second_derivative_central'] = data[source].rolling(period).apply(lambda x: tsf.mean_second_derivative_central(x), raw=True)
    
    feats['number_crossing__q_0.25']        = data[source].rolling(period).apply(lambda x: tsf.number_crossing_m(x, np.quantile(x, 0.25)), raw=True)
    feats['number_crossing__q_0.50']        = data[source].rolling(period).apply(lambda x: tsf.number_crossing_m(x, np.quantile(x, 0.5)), raw=True)
    feats['number_crossing__q_0.75']        = data[source].rolling(period).apply(lambda x: tsf.number_crossing_m(x, np.quantile(x, 0.75)), raw=True)
    feats['ratio_beyond_r_sigma__r_0.5']    = data[source].rolling(period).apply(lambda x: tsf.ratio_beyond_r_sigma(x, 0.5), raw=True)
    feats['ratio_beyond_r_sigma__r_1.0']    = data[source].rolling(period).apply(lambda x: tsf.ratio_beyond_r_sigma(x, 1.0), raw=True)
    feats['ratio_beyond_r_sigma__r_1.5']    = data[source].rolling(period).apply(lambda x: tsf.ratio_beyond_r_sigma(x, 1.5), raw=True)
    feats['ratio_beyond_r_sigma__r_2.0']    = data[source].rolling(period).apply(lambda x: tsf.ratio_beyond_r_sigma(x, 2.0), raw=True)
    feats['ratio_beyond_r_sigma__r_3.0']    = data[source].rolling(period).apply(lambda x: tsf.ratio_beyond_r_sigma(x, 3.0), raw=True)
    feats['ratio_beyond_r_sigma__r_4.0']    = data[source].rolling(period).apply(lambda x: tsf.ratio_beyond_r_sigma(x, 4.0), raw=True)
    feats['ratio_beyond_r_sigma__r_6.0']    = data[source].rolling(period).apply(lambda x: tsf.ratio_beyond_r_sigma(x, 6.0), raw=True)
    feats['ratio_beyond_r_sigma__r_8.0']    = data[source].rolling(period).apply(lambda x: tsf.ratio_beyond_r_sigma(x, 8.0), raw=True)
    feats['ratio_beyond_r_sigma__r_10.0']   = data[source].rolling(period).apply(lambda x: tsf.ratio_beyond_r_sigma(x, 10.0), raw=True)
    feats['count_above_mean']               = data[source].rolling(period).apply(lambda x: tsf.count_above_mean(x), raw=True)
    feats['count_below_mean']               = data[source].rolling(period).apply(lambda x: tsf.count_below_mean(x), raw=True)
    feats['first_location_of_minimum']      = data[source].rolling(period).apply(lambda x: tsf.first_location_of_minimum(x), raw=True)
    feats['first_location_of_maximum']      = data[source].rolling(period).apply(lambda x: tsf.first_location_of_maximum(x), raw=True)
    feats['first_location_of_minimum']      = data[source].rolling(period).apply(lambda x: tsf.first_location_of_minimum(x), raw=True)
    feats['first_location_of_maximum']      = data[source].rolling(period).apply(lambda x: tsf.first_location_of_maximum(x), raw=True)
    feats['longest_strike_above_mean']      = data[source].rolling(period).apply(lambda x: tsf.longest_strike_above_mean(x), raw=True)
    feats['longest_strike_below_mean']      = data[source].rolling(period).apply(lambda x: tsf.longest_strike_below_mean(x), raw=True)
    feats['has_duplicate_min']              = data[source].rolling(period).apply(lambda x: tsf.has_duplicate_min(x), raw=True)
    feats['has_duplicate_max']              = data[source].rolling(period).apply(lambda x: tsf.has_duplicate_max(x), raw=True)
    
    # feats['spkt_welch_density__coeff_1']    = data[source].rolling(period).apply(lambda x: list(tsf.spkt_welch_density(x, [{'coeff': 1}]))[0][1], raw=True)
    # feats['spkt_welch_density__coeff_2']    = data[source].rolling(period).apply(lambda x: list(tsf.spkt_welch_density(x, [{'coeff': 2}]))[0][1], raw=True)
    # feats['spkt_welch_density__coeff_3']    = data[source].rolling(period).apply(lambda x: list(tsf.spkt_welch_density(x, [{'coeff': 3}]))[0][1], raw=True)
    # feats['spkt_welch_density__coeff_4']    = data[source].rolling(period).apply(lambda x: list(tsf.spkt_welch_density(x, [{'coeff': 4}]))[0][1], raw=True)
    # feats['spkt_welch_density__coeff_5']    = data[source].rolling(period).apply(lambda x: list(tsf.spkt_welch_density(x, [{'coeff': 5}]))[0][1], raw=True)
    # feats['spkt_welch_density__coeff_6']    = data[source].rolling(period).apply(lambda x: list(tsf.spkt_welch_density(x, [{'coeff': 6}]))[0][1], raw=True)
    # feats['spkt_welch_density__coeff_7']    = data[source].rolling(period).apply(lambda x: list(tsf.spkt_welch_density(x, [{'coeff': 7}]))[0][1], raw=True)
    # feats['spkt_welch_density__coeff_8']    = data[source].rolling(period).apply(lambda x: list(tsf.spkt_welch_density(x, [{'coeff': 8}]))[0][1], raw=True)
    # feats['spkt_welch_density__coeff_9']    = data[source].rolling(period).apply(lambda x: list(tsf.spkt_welch_density(x, [{'coeff': 9}]))[0][1], raw=True)
    # feats['spkt_welch_density__coeff_10']   = data[source].rolling(period).apply(lambda x: list(tsf.spkt_welch_density(x, [{'coeff': 10}]))[0][1], raw=True)
    
    feats['fft_aggregated__aggtype_centroid'] = data[source].rolling(period).apply(lambda x: list(tsf.fft_aggregated(x, [{'aggtype': 'centroid'}]))[0][1], raw=True)
    feats['fft_aggregated__aggtype_variance'] = data[source].rolling(period).apply(lambda x: list(tsf.fft_aggregated(x, [{'aggtype': 'variance'}]))[0][1], raw=True)
    feats['fft_aggregated__aggtype_skew']     = data[source].rolling(period).apply(lambda x: list(tsf.fft_aggregated(x, [{'aggtype': 'skew'}]))[0][1], raw=True)
    feats['fft_aggregated__aggtype_kurtosis'] = data[source].rolling(period).apply(lambda x: list(tsf.fft_aggregated(x, [{'aggtype': 'kurtosis'}]))[0][1], raw=True)
    
    feats['cid_ce__normalize_true'] = data[source].rolling(period).apply(lambda x: tsf.cid_ce(x, True), raw=True)
    feats['cid_ce__normalize_false'] = data[source].rolling(period).apply(lambda x: tsf.cid_ce(x, False), raw=True)
    
    feats['binned_entropy__max_bins_1']    = data[source].rolling(period).apply(lambda x: tsf.binned_entropy(x, 1), raw=True)
    feats['binned_entropy__max_bins_2']    = data[source].rolling(period).apply(lambda x: tsf.binned_entropy(x, 2), raw=True)
    feats['binned_entropy__max_bins_3']    = data[source].rolling(period).apply(lambda x: tsf.binned_entropy(x, 3), raw=True)
    feats['binned_entropy__max_bins_4']    = data[source].rolling(period).apply(lambda x: tsf.binned_entropy(x, 4), raw=True)
    feats['binned_entropy__max_bins_5']    = data[source].rolling(period).apply(lambda x: tsf.binned_entropy(x, 5), raw=True)
    feats['binned_entropy__max_bins_6']    = data[source].rolling(period).apply(lambda x: tsf.binned_entropy(x, 6), raw=True)
    feats['binned_entropy__max_bins_7']    = data[source].rolling(period).apply(lambda x: tsf.binned_entropy(x, 7), raw=True)
    feats['binned_entropy__max_bins_8']    = data[source].rolling(period).apply(lambda x: tsf.binned_entropy(x, 8), raw=True)
    feats['binned_entropy__max_bins_9']    = data[source].rolling(period).apply(lambda x: tsf.binned_entropy(x, 9), raw=True)
    feats['binned_entropy__max_bins_10']   = data[source].rolling(period).apply(lambda x: tsf.binned_entropy(x, 10), raw=True)
    
    for m in [1, 2, 3, 4, 5]:
        for r in [1, 3, 10, 30, 100]:
            if m < period and r < period:
                1==1
                #feats['max_langevin_fixed_point__m_'+str(m)+'__r_'+str(r)] = data[source].rolling(period).apply(lambda x: tsf.max_langevin_fixed_point(x, m, r), raw=True)

    for num_seg in [1, 2, 3, 4, 5]:
        for seg_focus in [0, 1, 2, 3, 4]:
            if num_seg < period and r < period and seg_focus < num_seg:
                feats['energy_ratio_by_chunks__num_seg_'+str(num_seg)+'__seg_focus_'+str(seg_focus)] = data[source].rolling(period).apply(lambda x: tsf.energy_ratio_by_chunks(x, [{'num_segments': num_seg, 'segment_focus': seg_focus}]), raw=True)
    
    periods = [1, 2, 3, 4, 5, 6, 8, 10, 30, 60, 120, 180, 300, 600, 900, 1440]
    for _period in periods:
        if _period >= period:
            continue
        feats['time_reversal_asy_stat__lag_'+str(_period)]  = data[source].rolling(period).apply(lambda x: tsf.time_reversal_asymmetry_statistic(x, _period), raw=True)
        feats['partial_autocorrelation__lag_'+str(_period)] = data[source].rolling(period).apply(lambda x: tsf.partial_autocorrelation(x, [{'lag': _period}])[0][1], raw=True)
        feats['autocorrelation__lag_'+str(_period)]         = data[source].rolling(period).apply(lambda x: tsf.autocorrelation(x, _period), raw=True)
        feats['number_cwt_peaks__width_'+str(_period)]      = data[source].rolling(period).apply(lambda x: tsf.number_cwt_peaks(x, _period), raw=True)
        feats['c3__lag_'+str(_period)]                      = data[source].rolling(period).apply(lambda x: tsf.c3(x, _period), raw=True)
        feats['apprxent__strd_'+str(_period)+'__filvl_0.1'] = data[source].rolling(period).apply(lambda x: tsf.approximate_entropy(x, _period, 0.1), raw=True)
        feats['apprxent__strd_'+str(_period)+'__filvl_0.3'] = data[source].rolling(period).apply(lambda x: tsf.approximate_entropy(x, _period, 0.3), raw=True)
        feats['apprxent__strd_'+str(_period)+'__filvl_0.5'] = data[source].rolling(period).apply(lambda x: tsf.approximate_entropy(x, _period, 0.5), raw=True)
        feats['apprxent__strd_'+str(_period)+'__filvl_0.7'] = data[source].rolling(period).apply(lambda x: tsf.approximate_entropy(x, _period, 0.7), raw=True)
        feats['apprxent__strd_'+str(_period)+'__filvl_0.9'] = data[source].rolling(period).apply(lambda x: tsf.approximate_entropy(x, _period, 0.9), raw=True)
    
    return feats

In [ ]:
df['NUM_TRADES'].rolling(100).apply(lambda x: tsf.number_peaks(x, 3), raw=True)

In [27]:
strt = time.time()
feats = get_feats_tsf(df.iloc[:100000])
feats.shape
time.time() - strt

(100000, 103)

4756.74400305748

In [29]:
feats.apply(lambda x: hash(tuple(x.values,))).drop_duplicates().index.shape

(84,)